In [1]:
# import polars as pl
import re
from tqdm import tqdm
import pandas as pd
import sqlite3 as db

In [1]:
combined_data_files = [
    r"data/combined_data_1.txt",
    r"data/combined_data_2.txt",
    r"data/combined_data_3.txt",
    r"data/combined_data_4.txt"

]

results = []
current_film = None
con = db.connect('netflix.db')

for file in combined_data_files:

    with open(file, "r") as file:
        lines = file.readlines()
        for line in tqdm(lines):
            if re.match(r"\d*:", line):
                current_film = int(line.strip()[:-1])
            else:
                vals = line.split(",")
                user = int(vals[0])
                rating = int(vals[1])
                date = vals[2]
                results.append([current_film, user, rating, date])
    print("Creating dataframe")
    data = pd.DataFrame(results, columns=["film", "user", "rating", "date"])
    print("Saving to DB")
    data.to_sql('netflix_data', con, if_exists='append', index=False)
    results = []

100%|██████████| 22605786/22605786 [00:40<00:00, 561613.40it/s]


Creating dataframe
Saving to DB


100%|██████████| 26851926/26851926 [00:51<00:00, 526002.14it/s]


Creating dataframe
Saving to DB


In [4]:
con = db.connect('netflix.db')
# sql query that samples 1000 rows from the table
data = pd.read_sql_query("SELECT * FROM netflix_data ORDER BY RANDOM() LIMIT 100000", con)

In [5]:
con = db.connect('netflix_dev.db')
data.to_sql('netflix_data', con, if_exists='append', index=False)

100000

In [17]:
file_movies = r"data/movie_titles.csv"
results = []

with open(file_movies, "r") as file:
    lines = file.readlines()
    for line in tqdm(lines):
        vals = line.split(",", 2)
        film = int(vals[0])
        year = vals[1]
        title = vals[2]
        results.append([film, year, title])

100%|██████████| 17770/17770 [00:00<00:00, 1235377.28it/s]


In [24]:
df = pd.DataFrame(results, columns=["film", "year", "title"])
df['title'] = df['title'].str.strip()

con = db.connect('netflix_dev.db')

df.to_sql('movie_titles', con, if_exists='append', index=False)

17770

In [27]:
data = pd.read_sql_query("SELECT * FROM netflix_data", con)
data

,film,user,rating,date
0,12401,993438,2,2005-04-15\n
1,14103,328402,4,2005-04-17\n
2,13691,1555660,4,2004-11-15\n
3,11607,1281112,5,2005-05-05\n
4,11047,713840,4,2005-12-17\n
...,...,...,...,...
99995,4306,1999723,4,2005-10-10\n
99996,13748,536263,5,2005-05-18\n
99997,13517,21823,3,2003-09-25\n
99998,5894,1107428,4,2005-03-01\n
